### Test with the following JSON

```json
{
  "card_transactions": [
    {
    "transaction_id": "9876543210",
    "card_number": "4956965974959986",
    "transaction_datetime": "2015-06-01 00:00:00",
    "amount": 40.730000,
    "use_chip": "Chip Transaction",
    "merchant_city": "north oxford",
    "merchant_state": "ma",
    "zip": "1537",
    "mcc": "7538",
    "errors": "no_error"
    },
    {
    "transaction_id": "1234567890",
    "card_number": "4848774820047542",
    "transaction_datetime": "2025-06-01 00:00:00",
    "amount": 125.239998,
    "use_chip": "Chip Transaction",
    "merchant_city": "rome",
    "merchant_state": "italy",
    "zip": "0",
    "mcc": "5311",
    "errors": "no_error"
}
  ]
}
```

In [1]:
import redis
import json
import time
import uuid

# Connect to Redis
r = redis.Redis(host='redis', port=6379, db=1, decode_responses=False) # Keep decode_responses=False for raw bytes

def store_prediction(model_name: str, model_version: str, input_features: dict, predicted_value: float, actual_value: float = None):
    """
    Stores a single prediction result in Redis using a Hash and updates a Sorted Set for time-based indexing.
    """
    timestamp = int(time.time())
    prediction_id = str(uuid.uuid4()) # Unique ID for each prediction
    prediction_key = f"prediction:{timestamp}:{prediction_id}"

    # Prepare data for Redis Hash
    prediction_data = {
        "timestamp": timestamp,
        "model_name": model_name,
        "model_version": model_version,
        "input_features": json.dumps(input_features), # Store dict as JSON string
        "predicted_value": predicted_value,
        "actual_value": actual_value # Can be None if not available yet
    }

    # Convert all values to bytes for Redis
    prediction_data_bytes = {k.encode('utf-8'): str(v).encode('utf-8') for k, v in prediction_data.items()}
    if actual_value is None:
        prediction_data_bytes.pop(b'actual_value') # Don't store if None

    # Use a Redis pipeline for atomicity and efficiency
    with r.pipeline() as pipe:
        pipe.hset(prediction_key, mapping=prediction_data_bytes)
        pipe.zadd("predictions_by_time", {prediction_key: timestamp})
        pipe.execute()

    print(f"Stored prediction with ID: {prediction_key}")
    return prediction_key

def get_predictions_in_range(start_timestamp: int, end_timestamp: int):
    """
    Retrieves prediction results within a specified timestamp range.
    """
    # Get prediction keys from the Sorted Set based on the timestamp range
    # ZRANGESBYSSCORE returns members as bytes
    prediction_keys_bytes = r.zrangebyscore("predictions_by_time", start_timestamp, end_timestamp)

    if not prediction_keys_bytes:
        print(f"No predictions found between {start_timestamp} and {end_timestamp}.")
        return []

    print(f"Found {len(prediction_keys_bytes)} prediction keys in the range. Fetching details...")

    # Fetch details for each prediction key using a pipeline for efficiency
    with r.pipeline() as pipe:
        for key_bytes in prediction_keys_bytes:
            pipe.hgetall(key_bytes) # HGETALL returns hash fields as bytes
        prediction_details_bytes = pipe.execute()

    # Process and decode the results
    decoded_predictions = []
    for i, details_bytes in enumerate(prediction_details_bytes):
        decoded_item = {}
        for k_bytes, v_bytes in details_bytes.items():
            key_str = k_bytes.decode('utf-8')
            value_str = v_bytes.decode('utf-8')

            # Special handling for numerical values and JSON string
            if key_str == 'timestamp':
                decoded_item[key_str] = int(value_str)
            elif key_str in ['predicted_value', 'actual_value']:
                decoded_item[key_str] = float(value_str)
            elif key_str == 'input_features':
                decoded_item[key_str] = json.loads(value_str)
            else:
                decoded_item[key_str] = value_str
        decoded_predictions.append(decoded_item)

    return decoded_predictions


In [4]:
import pytz
from datetime import datetime

# Use pytz to handle timezone-aware datetime strings
def get_unix_timestamp_with_tz(datetime_str, tz_str='UTC'):
    tz = pytz.timezone(tz_str)
    dt = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    dt = tz.localize(dt)  # Localize to the specified timezone
    return int(dt.timestamp())

# Example usage of get_unix_timestamp_with_tz
datetime_str = "2025-06-16 19:20:00"
# Detect local timezone and get unix timestamp

unix_timestamp = get_unix_timestamp_with_tz(datetime_str, 'Asia/Singapore')
print(f"Unix timestamp for '{datetime_str}' in Asia/Singapore: {unix_timestamp}")

r = redis.Redis(host='redis', port=6379, db=1, decode_responses=False) # Keep decode_responses=False for raw bytes

start_range = unix_timestamp - 3600 # Last 1 hr
end_range = unix_timestamp

print(f"Searching for predictions between {start_range} and {end_range}")
results = get_predictions_in_range(start_range, end_range)
print(results)

Unix timestamp for '2025-06-16 19:20:00' in Asia/Singapore: 1750072800
Searching for predictions between 1750069200 and 1750072800
Found 2 prediction keys in the range. Fetching details...
[{'transaction_id': '1234567890', 'is_fraud': '1'}, {'transaction_id': '9876543210', 'is_fraud': '0'}]
